In [1]:
from bs4 import BeautifulSoup
import requests

/Users/saumyasadhsaumya/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
import requests
from bs4 import BeautifulSoup
import csv
import os 

static_url = "https://www.ustatesloans.org/law/"
states = [
    'WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'AZ', 'NM', 
    'CO', 'ND', 'SD', 'NE', 'KS', 'OK', 'TX', 'MN', 'IA', 'MO', 
    'AR', 'LA', 'WI', 'IL', 'IN', 'MI', 'OH', 'KY', 'TN', 'MS', 
    'AL', 'GA', 'FL', 'SC', 'NC', 'VA', 'WV', 'PA', 'NY', 'VT', 
    'NH', 'ME', 'MA', 'RI', 'CT', 'NJ', 'DE', 'MD', 'DC', 'AK', 
    'HI', 'GU', 'PR', 'VI'
] #list of states as seen in the url 

state_full_names = {
    'WA': 'Washington', 'OR': 'Oregon', 'CA': 'California', 'NV': 'Nevada', 
    'ID': 'Idaho', 'MT': 'Montana', 'WY': 'Wyoming', 'UT': 'Utah', 'AZ': 'Arizona', 
    'NM': 'New Mexico', 'CO': 'Colorado', 'ND': 'North Dakota', 'SD': 'South Dakota', 
    'NE': 'Nebraska', 'KS': 'Kansas', 'OK': 'Oklahoma', 'TX': 'Texas', 'MN': 'Minnesota', 
    'IA': 'Iowa', 'MO': 'Missouri', 'AR': 'Arkansas', 'LA': 'Louisiana', 'WI': 'Wisconsin', 
    'IL': 'Illinois', 'IN': 'Indiana', 'MI': 'Michigan', 'OH': 'Ohio', 'KY': 'Kentucky', 
    'TN': 'Tennessee', 'MS': 'Mississippi', 'AL': 'Alabama', 'GA': 'Georgia', 'FL': 'Florida', 
    'SC': 'South Carolina', 'NC': 'North Carolina', 'VA': 'Virginia', 'WV': 'West Virginia', 
    'PA': 'Pennsylvania', 'NY': 'New York', 'VT': 'Vermont', 'NH': 'New Hampshire', 
    'ME': 'Maine', 'MA': 'Massachusetts', 'RI': 'Rhode Island', 'CT': 'Connecticut', 
    'NJ': 'New Jersey', 'DE': 'Delaware', 'MD': 'Maryland', 'DC': 'District of Columbia', 
    'AK': 'Alaska', 'HI': 'Hawaii', 'GU': 'Guam', 'PR': 'Puerto Rico', 'VI': 'Virgin Islands'
} #mapping state code to full form, so it finds the right state on the website 

with open("payday_loan_data.txt", "w") as file: #to store the text based result 
    for state in states:
        state_name = state_full_names[state]
        url = f"{static_url}{state.lower()}/"
        response = requests.get(url)  #contruct the full URL and send GET request to fetch webpage for the state

        if response.status_code == 200: #200 is what you should look for, if the request is sucessful, parses the HTML using beautifulsoup 
            soup = BeautifulSoup(response.text, 'html.parser')

            file.write(f"State: {state_name}\n")
            file.write("=" * 40 + "\n") #writes the state's name and sperators line to the textfile 

            regulations_section = soup.find('div', class_='card mb-5') #find the div that contains payday loan regulations 
            if regulations_section:
                file.write("\nPayday Loan Regulations:\n")
                rows = regulations_section.find_all('div', class_='row border-top') #If found, it loops through rows (<div class='row border-top'>), extracts key (e.g., "Maximum Loan Amount") and value (e.g., "$700").

                for row in rows:
                    key = row.find('strong')
                    value = row.find('div', class_='col-md-7 py-1')
                    if key and value:
                        file.write(f"{key.get_text(strip=True)}: {value.get_text(strip=True)}\n") #writes the regulations in text file 
            else:
                file.write("No regulations section found.\n") 

            data_dir = "scraped_data/"

            os.makedirs(data_dir, exist_ok=True)

            statistics_header = soup.find('h2', string=lambda text: text and "Statistics" in text) #to extract the statistics table, find header and extract statistics seciton
            if statistics_header:
                statistics_table = statistics_header.find_next('figure', class_='wp-block-table')
                if statistics_table:
                    csv_file_path = os.path.join(data_dir, f"{state}_statistics.csv")

                    with open(csv_file_path, mode='w', newline='') as csv_file:
                        csv_writer = csv.writer(csv_file)

                        header_row = statistics_table.find('thead') #writes the table header in the CSV file 
                        if header_row:
                            headers = [header.get_text(strip=True) for header in header_row.find_all('th')]
                            csv_writer.writerow(headers)  

                        rows = statistics_table.find_all('tr')
                        for row in rows:
                            columns = [col.get_text(strip=True) for col in row.find_all('td')]
                            if columns:
                                csv_writer.writerow(columns)

                    print(f"Statistics table saved to {csv_file_path}")
                else:
                    print(f"No statistics table found for {state_name}.")
            else:
                print(f"No statistics section found for {state_name}.")

            history_header = soup.find('h2', string=lambda text: text and "The History of Payday Loans" in text) #extract history of payday loan 
            if history_header:
                file.write("\nHistory of Payday Loans:\n")
                
                history_list = history_header.find_next('ul', class_='list-group wp-block-list')
                if history_list:
                    items = history_list.find_all('li')
                    for item in items:
                        date = item.find('strong')
                        event = item.get_text(strip=True)
                        if date:
                            event = event.replace(date.get_text(strip=True), '').strip()
                            file.write(f"{date.get_text(strip=True)}: {event}\n")
                        else:
                            file.write(f"- {event}\n")
                else:
                    file.write("No history content found.\n")
            else:
                file.write(f"No history section found for {state_name}.\n")

            file.write("\n" + "=" * 40 + "\n\n") #finalise the state data 
        else:
            file.write(f"Failed to fetch data for {state_name} (status code: {response.status_code})\n")


No statistics section found for Washington.
No statistics section found for Oregon.
Statistics table saved to scraped_data/CA_statistics.csv
No statistics section found for Nevada.
No statistics section found for Idaho.
No statistics section found for Montana.
No statistics section found for Wyoming.
Statistics table saved to scraped_data/UT_statistics.csv
No statistics section found for Arizona.
No statistics section found for New Mexico.
Statistics table saved to scraped_data/CO_statistics.csv
No statistics section found for North Dakota.
No statistics section found for South Dakota.
Statistics table saved to scraped_data/NE_statistics.csv
Statistics table saved to scraped_data/KS_statistics.csv
Statistics table saved to scraped_data/OK_statistics.csv
Statistics table saved to scraped_data/TX_statistics.csv
No statistics section found for Minnesota.
No statistics section found for Iowa.
No statistics section found for Missouri.
No statistics section found for Arkansas.
No statistics 

In [43]:
import pandas as pd

In [44]:
data = pd.read_csv("scraped_data/CA_statistics.csv")
data

,Year,No. of Stores,"No. of Clients,million","No. of Loans,million","Value of Fees,million","Value of Loans,billion"
0,2010,"2,144",1.6,12.1,–,$3.1
1,2011,"2,119",1.7,12.4,–,$3.3
2,2012,"2,100",1.8,12.3,–,$3.2
3,2013,"2,058",1.8,12.2,–,$3.2
4,2014,"2,014",1.8,12.4,–,$3.4
5,2015,"1,969",1.9,12.3,–,$4.2
6,2016,"1,854",1.8,11.5,$467.2,$3.1
7,2017,"1,705",1.7,10.7,$436.4,$2.9
8,2018,"1,645",1.6,10.2,$420.5,$2.8
9,2019,"1,551",1.6,10.2,$418.4,$2.8
